### TASKS 
Concrètement on a un problème de classification : pour une série de données en entrées on aimerait savoir si une personne est vaccinée ou non et pour se faire on va entraîner des modèles sur tout un tas de données d'entraînement.
#### 1) Explorer les données 
Se poser des questions comme combien de features on a , quelle nature (string, int, catégorique ou pas, ...) , quelle lien avec les labels de prédictions (déterminer si il y a corrélation)
#### 2) Normaliser les données
-data imputation --> remplacer les valeurs vides (NaN) par une autre (par ex. par la moyenne de la colonne), ou supprimer l'entrée <br> 

-categorical encoding --> des catégories hiérarchiques comme "Below Poverty" ou "<= 75,000, Above Poverty" vont être encodées par des ints (0 = pauvre, 1 = juste au dessus, ... 3 = riche) <br> 

-one-hot-encoding --> les catégories non hiérarchiques comme les régions géographiques vont être extraites et former de nouvelles catégories où les entrées seront binaires (0 les gens ne sont pas de là, 1 ils sont de là) <br> 

-normalisation numérique --> chaque i dans une colonne va être remplacée par (i-µ)/sd , µ = moyenne de la colonne , sd = deviation standard 
<br>
#### 3) FAIRE JOUJOU AVEC DES MOD7LES 
Les modèles qui seront surement les plus rapides & les plus efficaces pour notre problème sont XG BOOST et Random Forest, tous deux des algorithmes basés sur les arbres décisionnels <br>
Je te conseille cette chaîne youtube qui est phé no mé nale pour comprendre les quelques bases théoriques de ces modèles<br>
https://www.youtube.com/watch?v=7VeUPuFGJHk

https://www.youtube.com/watch?v=GrJP9FLV3FE

https://www.youtube.com/watch?v=J4Wdy0Wc_xQ

On benchmarkera ces modèles à l'aide de confusion matrix principalement : https://machinelearningmastery.com/confusion-matrix-machine-learning/


# Data Exploration

In [9]:
import pandas as pd
import numpy as np

PATH = '../Data/'
features_file = "training_set_features.csv"
label_file = "training_set_labels.csv"

feature_df = pd.read_csv(f"{PATH}{features_file}")
label_df = pd.read_csv(f"{PATH}{label_file}")

In [40]:
feature_df

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN
26705,26705,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Married,Rent,Employed,lrircsnp,Non-MSA,1.0,0.0,fcxhlnwr,haliazsg


In [41]:
# data type of each column/feature
feature_df.dtypes

respondent_id                    int64
h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                      

In [42]:
# for each column of our dataframe, count the number of NaN (empty) cells and put this number in the list
na_count = [(column,feature_df[column].isna().sum()) for column in feature_df.columns] 

print(na_count)


#nb la manière d'écrire ça en une ligne ça revient à faire ça : 
# na_count = []
# for column in feature_df.columns:
#      na_count.append((column,feature_df[column].isna().sum())))
#
#
# tu peux accéder à une colonne (feature) d'un dataframe en utilisant dataframe["colonne"] ou dataframe.colonne
# il y a pas mal de fonctions spécifiques à pandas qui te facilite la vie, ici isna() renvoie le nombre d'élément nul dans la colonne
# 
# en plus pandas est une librairie écrite en C donc ça va énormément plus vite d'apprendre à utiliser ces fonctions que
# d'hardcoder tout ça en python

[('respondent_id', 0), ('h1n1_concern', 92), ('h1n1_knowledge', 116), ('behavioral_antiviral_meds', 71), ('behavioral_avoidance', 208), ('behavioral_face_mask', 19), ('behavioral_wash_hands', 42), ('behavioral_large_gatherings', 87), ('behavioral_outside_home', 82), ('behavioral_touch_face', 128), ('doctor_recc_h1n1', 2160), ('doctor_recc_seasonal', 2160), ('chronic_med_condition', 971), ('child_under_6_months', 820), ('health_worker', 804), ('health_insurance', 12274), ('opinion_h1n1_vacc_effective', 391), ('opinion_h1n1_risk', 388), ('opinion_h1n1_sick_from_vacc', 395), ('opinion_seas_vacc_effective', 462), ('opinion_seas_risk', 514), ('opinion_seas_sick_from_vacc', 537), ('age_group', 0), ('education', 1407), ('race', 0), ('sex', 0), ('income_poverty', 4423), ('marital_status', 1408), ('rent_or_own', 2042), ('employment_status', 1463), ('hhs_geo_region', 0), ('census_msa', 0), ('household_adults', 249), ('household_children', 249), ('employment_industry', 13330), ('employment_occupa

We see that almost all features have some empty (NaN) cells, how to manage that ? It's the first step into cleaning our input datas. 

For the numerical features, what is often done is to replace the NaN values with the average value. Which average value ? Naive approach : average of the whole column. Other approach : instead of replacing by the mean of the whole column what we can do is clustering the values with another feature. 
By example : replacing the empty numerical values of *h1n1_concern* with the average values of *h1n1_concern* which have the same *income_poverty* or the same *h1n1_knowledge* ! 
It requires to make assumptions which might be correct or not.


For the categorical values , it's trickier and will depends on their nature and the number of NaN value we have.

In [44]:
# let's count the number of unique values for each categorical feature
unique_count = [(column,feature_df[column].nunique()) for column in feature_df.columns if feature_df.dtypes[column] == 'O']
print(unique_count)

[('age_group', 5), ('education', 4), ('race', 4), ('sex', 2), ('income_poverty', 3), ('marital_status', 2), ('rent_or_own', 2), ('employment_status', 3), ('hhs_geo_region', 10), ('census_msa', 3), ('employment_industry', 21), ('employment_occupation', 23)]
